In [1]:
import pandas as pd   # COMMON COMMAND
import numpy as np


from sklearn.model_selection import GroupKFold


import gc
import xgboost as xgb

model retrain for ordering

[code from here](https://www.kaggle.com/code/redeyes0007/try-one-ubiquent/edit)
there is model 

In [2]:
df = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet') ## COMMON COMMAND
df=df[:1500000]
features = [f'f_{i}' for i in range(300)]
for i in features:      # now, changing the data type of (f_0 -f_299) into (float32-float16) because that was fast to use read&Write.
    df[i]=df[i].astype(np.float16)
gc.collect()
few_features=['investment_id']   # remove row_id
features=few_features+features

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[features], df['target'], test_size=0.1, random_state=42)
del  df
gc.collect()

model = xgb.XGBRegressor(
    n_estimators=800,
    learning_rate=0.03,
    max_depth=12,
    subsample=0.95,
    colsample_bytree=0.9,
    #colsample_bylevel=0.75,
    missing=-999,
    random_state=1,
    tree_method='gpu_hist'  
    )
    
gc.collect()
    

# model_predict=model.predict(x_test)
model_xgb_2=xgb.Booster()
model_1=model_xgb_2.load_model('../input/train-model-1/model_2.model')
model.fit(X_train, y_train, early_stopping_rounds=20, eval_set=[(X_test, y_test)], verbose=1,xgb_model=model_1)

del X_train, y_train,X_test, y_test
gc.collect()

import gc
gc.collect()


[0]	validation_0-rmse:1.05608
[1]	validation_0-rmse:1.04820
[2]	validation_0-rmse:1.04062
[3]	validation_0-rmse:1.03348
[4]	validation_0-rmse:1.02675
[5]	validation_0-rmse:1.02033
[6]	validation_0-rmse:1.01421
[7]	validation_0-rmse:1.00843
[8]	validation_0-rmse:1.00294
[9]	validation_0-rmse:0.99773
[10]	validation_0-rmse:0.99283
[11]	validation_0-rmse:0.98820
[12]	validation_0-rmse:0.98379
[13]	validation_0-rmse:0.97963
[14]	validation_0-rmse:0.97561
[15]	validation_0-rmse:0.97185
[16]	validation_0-rmse:0.96825
[17]	validation_0-rmse:0.96487
[18]	validation_0-rmse:0.96163
[19]	validation_0-rmse:0.95860
[20]	validation_0-rmse:0.95571
[21]	validation_0-rmse:0.95288
[22]	validation_0-rmse:0.95030
[23]	validation_0-rmse:0.94791
[24]	validation_0-rmse:0.94557
[25]	validation_0-rmse:0.94338
[26]	validation_0-rmse:0.94132
[27]	validation_0-rmse:0.93922
[28]	validation_0-rmse:0.93731
[29]	validation_0-rmse:0.93553
[30]	validation_0-rmse:0.93387
[31]	validation_0-rmse:0.93227
[32]	validation_0-

0

In [3]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

In [4]:
for (test_df, sample_prediction_df) in iter_test:
#     display(test_df)
    x_tt = test_df.loc[:, features]

    #y_pred = model.predict(cudf.from_pandas(pd.DataFrame([x_test])).astype('float32'))
    #y_pred = model.predict(pd.DataFrame([x_test]))[0]
    xgb_pred = model.predict(x_tt)
    sample_prediction_df['target'] = xgb_pred
    env.predict(sample_prediction_df) 
#     display(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
